In [1]:
import numpy as np 
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import glob
from tqdm import tqdm
from collections import OrderedDict, defaultdict
from IPython.display import clear_output
import time
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num, seq_len):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(seq_len)
        
    def forward(self,inputs):
        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [4]:
n_hidden = 128
n_joints = 25*2
n_categories = 2
n_layer = 3
seq_len = 30
model = LSTM(n_joints,n_hidden,n_categories,n_layer, seq_len).to(device)


model.load_state_dict(torch.load('rnn_train_1_dec7.pth'))
model.eval()

LSTM(
  (lstm): LSTM(50, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (bn): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [5]:
# test_fn = "/home/cgladu/activitynet-main/clips_eating/extracted_points/ns2_18.mp4_points.txt" 
test_fn = "/home/cgladu/activitynet-main/non_eating_clips/extracted_points/ns1_0_points.txt"

with open(test_fn, 'r') as file:
    lines = np.loadtxt(file, dtype=str)
    ret=[]
    for line in lines:
        frame=[]
        vals = line.split(',')
        for val in vals:
            convert = float(val)
            frame.append(convert)
        ret.append(frame)
    r = np.array(ret)

r.shape

(3461, 50)

In [6]:
Xt=r[:seq_len]
Xt.shape

(30, 50)

In [7]:
Xt=torch.from_numpy(Xt).float().to(device)
Xt=Xt.reshape(1, -1, 50)
Xt.shape

torch.Size([1, 30, 50])

In [8]:
logits=model(Xt)
po=logits.argmax(axis=1)
po

tensor([0], device='cuda:0')